In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def create_soup(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

def print_news(index, title, link):
    print("{}. {}".format(index+1, title))
    print("링크 : {}".format(link))
    
def scrape_english():
    print("[오늘의 영어 회화]")
    print()
    url = "https://www.hackers.co.kr/?c=s_eng/eng_contents/I_others_english"
    soup = create_soup(url)
    
    sentences = soup.find_all('div', attrs={'id':re.compile('^conv_kor_t')})
    
#     print(len(sentences)//2)
    
    print('- 영어 지문 -')
    for sentence in sentences[len(sentences)//2:]: # 8문장이라고 가정, 4~7이 영어, 0~3이 한글
        print(sentence.get_text().strip())
        
    print()
    print('- 한글 지문 -')
    for sentence in sentences[:len(sentences)//2]:
        print(sentence.get_text().strip())
    print()

def scrape_headline_news():
    print("[최신 뉴스]")
    url = "https://news.naver.com/"
    soup = create_soup(url)
    
    news = soup.find('div', class_='cjs_channel_card')
    news_list = news.find_all('div', class_='cjs_journal_wrap _item_contents', limit=3)
    
    for index, news in enumerate(news_list):
        title = news.find('div', class_='cjs_t').get_text().strip()
        link = news.find('a', class_='cjs_news_a').get('href')
        
        print_news(index, title, link)

    print()
    
    
def scrape_it_news():
    print("[IT 뉴스]")
    url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230"
    soup = create_soup(url)
    
    news_list = soup.find('ul', class_='type06_headline').find_all('li', limit=3)
    for index, news in enumerate(news_list):
        a = news.select('ul.type06_headline > li > dl > dt.photo > a > img')
        title = a[0].attrs['alt']
        link = news.find('a').get('href')
        print_news(index, title, link)
    
    print()

def scrape_weather():
    print("[오늘의 날씨]")
    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%84%9C%EC%9A%B8+%EB%82%A0%EC%94%A8"
    soup = create_soup(url)
    
    # 맑음, 어제보다 00 높아요
    weather = soup.find('span', class_="weather before_slash").get_text()
    cast = soup.find("p", attrs={'class':'summary'}).get_text().replace("맑음","")
    
    # 현재 00도 (최저 00 / 최고 00)
    curr_temp = soup.find('div', class_='temperature_text').get_text().strip()
    curr_temp = curr_temp.replace("현재 온도", "현재 온도: ")
    min_temp = soup.find('span', class_='lowest').get_text().replace("최저기온","최저 기온: ")
    max_temp = soup.find('span',class_='highest').get_text().replace("최고기온","최고 기온: ")
    
    # 강수 확률
    moring_rain_rate = soup.select_one('div > div.list_box > ul > li:nth-child(1) > div > div.cell_weather > span:nth-child(1) > span').get_text().strip()
    afternoon_rain_rate = soup.select_one('div > div.list_box > ul > li:nth-child(1) > div > div.cell_weather > span:nth-child(2) > span').get_text().strip()
    
    # 미세 먼지
    #똑같은 태그에 여러 클래스가 존재하면 리스트 형식으로 받으면 된다. 
    #클래스, 아이디, 속성 등으로 여러개 찾고 싶을때는 {'class':'', 'id':''} 식으로 찾음
    #특정 태그의 특성을 완벽히 찾고 싶으면 {'class':''}, text='' 이런식으로 구분자를 주면 됨
    dust = soup.find('ul', class_='today_chart_list')
    pm10 = dust.find_all('li')[0].get_text().strip().replace("미세먼지", "미세먼지: ")
    pm25 = dust.find_all('li')[1].get_text().strip().replace("초미세먼지", "초미세먼지: ")
    print(weather + ", "+cast)
    print(curr_temp)
    print(min_temp)
    print(max_temp)
    print("강수 확률: " + moring_rain_rate)
    print("강수 확률: " + afternoon_rain_rate)
    print(pm10)
    print(pm25)
    
    

In [4]:
if __name__ == "__main__":
    scrape_weather()
    print()
    scrape_headline_news()
    print()
    scrape_it_news()
    print()
    scrape_english()

[오늘의 날씨]
맑음, 어제보다 0° 높아요   
현재 온도: -1°
최저 기온: -6°
최고 기온: 4°
강수 확률: 오전 0%
강수 확률: 오후 0%
미세먼지:  보통
초미세먼지:  나쁨

[최신 뉴스]
1. 비트코인 5100만원대로↓ 이더리움 400만원대 붕괴…계속되는 코인 하락세
링크 : https://n.news.naver.com/article/243/0000021102
2. 차선 안 비켜줬다고 인도까지 넘어가며 급제동·보복운전…차주는 "깜빡이 켰잖아" (영상)
링크 : https://n.news.naver.com/article/119/0002565012
3. 보디빌더 대회 4등 女…"前남편 조롱에 자극 받아, 現애인 15살 연하 "
링크 : https://n.news.naver.com/article/003/0010933201


[IT 뉴스]
1. “면보다 스프 먼저 넣는게 더 맛있다?” 라면맛 과학적 근거가
링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=016&aid=0001934790
2. 카카오 차기대표 ‘먹튀 논란’에 직원 1800여명 “류영준 반대”
링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=293&aid=0000037580
3. 국내 규제 막혀 해외로…'뉴코애드윈드', DH 손잡고 한국 뜬다
링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=293&aid=0000037579


[오늘의 영어 회화]

- 영어 지문 -
Mr. Kim: We need to know how long it will take you to deliver the products.
Sarah: I cannot give you an exact answer at th